In [ ]:
from cimloader.databases import ConnectionParameters
from cimloader.databases.mysql import MySQLConnection

In [ ]:
params = ConnectionParameters(host= "localhost", database="rc4_2021", username="root", password="password", cim_profile='rc4_2021', namespace="http://iec.ch/TC57/CIM100#")
mysql = MySQLConnection(params)

In [ ]:
mysql.configure()

Unable to create table for class DateInterval
Unable to create table for class DecimalQuantity
Unable to create table for class Limit
Unable to create table for class MonthDayInterval
Unable to create table for class Procedure
Unable to create table for class StateVariable


In [ ]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [ ]:
from cimgraph.databases import Parameter, ConnectionParameters
from cimgraph.databases.blazegraph.blazegraph import BlazegraphConnection
from cimgraph.models import FeederModel
import json
import time
# Blazegraph connection for running outside the container
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile='rc4_2021')
bg = BlazegraphConnection(params, 'rc4_2021')


#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
# feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
# feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node
## Create Distributed CIM Model:
feeder = cim.Feeder(mRID=feeder_mrid)
start = time.perf_counter()
network_bg = FeederModel(connection=bg, container=feeder, distributed=False, cim_profile = 'rc4_2021')
build_time = time.perf_counter() - start
print("blazegraph init time:", build_time)


cim_class = cim.ACLineSegment
start = time.perf_counter()
network_bg.get_all_edges(cim_class)
build_time = time.perf_counter() - start
print("blazegraph query time for", len(network_bg.graph[cim_class]), "objects:", build_time)

blazegraph init time: 0.3786984649996157
blazegraph query time for 3909 objects: 2.756139802000689


In [ ]:
mysql.upload_from_cimgraph(network_bg)

In [ ]:
start = time.perf_counter()
myresult = mysql.execute("SELECT * FROM ACLineSegment WHERE _mRID in ('_173DFB74-F387-4F0A-8A8E-447ED35110B1')")


for x in myresult:
    print(x)
build_time = time.perf_counter() - start
print("mysql query time:", build_time)

('_173DFB74-F387-4F0A-8A8E-447ED35110B1', None, None, 'tpx2001208c0', None, None, None, None, None, '{"@id": "_13AFD6B9-C5ED-4F3D-8B64-8DC2943C5F6A", "@type": "Location"}', '["{\\"@type\\": \\"Analog\\", \\"@id\\": \\"_9c8c5045-6662-47d0-af4a-bf54bdb11128\\"}", "{\\"@type\\": \\"Analog\\", \\"@id\\": \\"_ccd6fd93-35db-4a3d-a17c-4b6b3abe2702\\"}", "{\\"@type\\": \\"Analog\\", \\"@id\\": \\"_29a577b6-057f-48f1-a0e1-c52ba6d0b72d\\"}", "{\\"@type\\": \\"Analog\\", \\"@id\\": \\"_4a5da26f-7bb9-4be5-943b-6e4c961b99ac\\"}"]', None, None, None, None, None, None, None, None, '{"@id": "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA", "@type": "Feeder"}', None, None, None, '{"@id": "_8FCD8F92-D13C-436C-9C8C-15B6121DEC70", "@type": "BaseVoltage"}', None, '["{\\"@type\\": \\"Terminal\\", \\"@id\\": \\"_7B62FE02-465D-43A8-BF62-7293C4F9EE97\\"}", "{\\"@type\\": \\"Terminal\\", \\"@id\\": \\"_41442059-44B8-4105-8D41-638EFE02CCAF\\"}"]', 15.24, None, None, None, None, None, None, None, None, None, '["{\\"@type\

In [ ]:
start = time.perf_counter()
myresult = mysql.execute("""SELECT * FROM ACLineSegment WHERE _EquipmentContainer->'$."@id"' = '_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA' """)


# for x in myresult:
#     print(x)
build_time = time.perf_counter() - start
print("mysql query time for", len(myresult), " rows in", build_time)

mysql query time for 3909  rows in 0.021856936000403948
